# ARIMA aplicado na serie temporal 01
## Cotacao do Dolar em relacao ao Real

In [ ]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA

# imports de modulos que estao definidos dentro do projeto
import data
import timeseries
import arima

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
PORCENTAGEM: float = 0.86

## Transformacao da serie temporal 'MENSAL com duas amostras'

In [ ]:
df_monthly2 = data.load_timeseries01_monthly2()
#timeseries.plot_timeserie(df_monthly2)

In [ ]:
ts_transform = timeseries.ts_transform1(df_monthly2)
#timeseries.plot_timeserie(ts_transform['ts_log'])

In [ ]:
timeseries.check_stationarity(ts_transform['ts_log'], window=24)

In [ ]:
#timeseries.plot_timeserie(ts_transform['ts_log_diff'])

In [ ]:
timeseries.check_stationarity(ts_transform['ts_log_diff'], window=24)

## ARIMA aplicado na serie temporal 'MENSAL com duas amostras'

In [ ]:
train, test = arima.split_dataset(ts_transform['ts_log'], porcentagem=PORCENTAGEM, debug=True)
timeseries.plot_train_and_test(train, test)

In [ ]:
# melhores parametros encontrados no Grid Search
arima_params = (0, 1, 1)

model = ARIMA(train, order=arima_params)
results_ARIMA = model.fit(disp=-1)

train_size = int(len(ts_transform['ts_log_diff']) * PORCENTAGEM)
ts_train = ts_transform['ts_log_diff'][0:train_size]

plt.plot(ts_train.index.to_pydatetime(), ts_train.values)
plt.plot(ts_train.index.to_pydatetime(), results_ARIMA.fittedvalues, color='red')
plt.title('MSE: %.9f' % mean_squared_error(ts_train.values, results_ARIMA.fittedvalues.values))

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)

predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(train.iloc[0], index=train.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)

predictions_ARIMA = np.exp(predictions_ARIMA_log)
predictions_ARIMA = predictions_ARIMA.apply(lambda x: x*1.8)
predictions_ARIMA = predictions_ARIMA[1:]
#print(len(predictions_ARIMA))
#print(predictions_ARIMA)

train_size = int(len(df_monthly2) * PORCENTAGEM)
ts_train = df_monthly2[1:train_size]
#print(len(ts_train))
#print(ts_train)

plt.plot(ts_train.index.to_pydatetime(), ts_train.values)
plt.plot(ts_train.index.to_pydatetime(), predictions_ARIMA.values)
#plt.title('RMSE: %.9f' % np.sqrt(sum((predictions_ARIMA - ts_train)**2) / len(ts_train)))
plt.title('MSE: %.9f'% mean_squared_error(ts_train.values, predictions_ARIMA.values))

In [ ]:
predictions = list()
historical = list()

model = ARIMA(train, order=arima_params)
model_fit = model.fit(disp=0)

t = 0
for yhat in forecast:
    predicted = np.exp(float(yhat))
    predictions.append(predicted)
    
    observed =  np.exp(test['rate'][t])
    historical.append(observed)
    print('Predicted = %.9f, Expected = %.9f' % (predicted, observed))
    t += 1

error = mean_squared_error(historical, predictions)
print('Test MSE: %.9f' % error)

plt.plot(historical)
plt.plot(predictions, color='red')
plt.show()

## Transformacao da serie temporal 'MENSAL'

In [ ]:
df_monthly = data.load_timeseries01_monthly()
#timeseries.plot_timeserie(df_monthly)

In [ ]:
ts_transform = timeseries.ts_transform1(df_monthly)
timeseries.plot_timeserie(ts_transform['ts_log'])

In [ ]:
timeseries.check_stationarity(ts_transform['ts_log'], window=12)

In [ ]:
timeseries.plot_timeserie(ts_transform['ts_log_diff'])

In [ ]:
timeseries.check_stationarity(ts_transform['ts_log_diff'], window=12)

## ARIMA aplicado na serie temporal 'MENSAL'

In [ ]:
# teste com 150 combinacoes de parametros
# melhores parametros encontrados no Grid Search
# Melhores parametros: ARIMA(0, 1, 1) MSE=0.001153267
arima_params = (0, 1, 1)